In [ ]:
from keras.models import Sequential
from keras.layers import Conv3D, MaxPooling3D, Flatten, Dense
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Conv3DTranspose, BatchNormalization, LeakyReLU, concatenate
from keras.models import Sequential, Model
from keras.optimizers import Adam

# Define the input shape
input_shape = (160, 160, 192, 1)

# Create a sequential model
model = Sequential()

# Add convolutional layers with max pooling
model.add(Conv3D(32, (3, 3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Conv3D(64, (3, 3, 3), activation='relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Conv3D(128, (3, 3, 3), activation='relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

# Add fully connected layers
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Define the input shape
img_shape = (160, 160, 192, 2)
latent_dim = 100

# Create a generator model
generator = Sequential()

generator.add(Dense(128 * 20 * 20 * 24, activation="relu", input_dim=latent_dim))
generator.add(Reshape((20, 20, 24, 128)))
generator.add(Conv3DTranspose(128, kernel_size=(3, 3, 3), strides=(2, 2, 2), padding="same"))
generator.add(BatchNormalization(momentum=0.8))
generator.add(LeakyReLU(alpha=0.2))
generator.add(Conv3DTranspose(64, kernel_size=(3, 3, 3), strides=(2, 2, 2), padding="same"))
generator.add(BatchNormalization(momentum=0.8))
generator.add(LeakyReLU(alpha=0.2))
generator.add(Conv3DTranspose(2, kernel_size=(3, 3, 3), strides=(2, 2, 2), padding="same", activation='tanh'))

# Create a discriminator model
discriminator = Sequential()

discriminator.add(Conv3D(32, kernel_size=(3, 3, 3), strides=(2, 2, 2), input_shape=img_shape, padding="same"))
discriminator.add(LeakyReLU(alpha=0.2))
discriminator.add(Dropout(0.25))
discriminator.add(Conv3D(64, kernel_size=(3, 3, 3)))


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

input_shape = (1, 182, 218, 182)

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()

        self.convnet = nn.Sequential(
            nn.Conv3d(1, 32, kernel_size=(3, 3, 3), padding=1),
            nn.MaxPool3d(kernel_size=(2, 2, 2)),
            nn.Conv3d(32, 64, kernel_size=(3, 3, 3), padding=1),
            nn.MaxPool3d(kernel_size=(2, 2, 2)),
            nn.Conv3d(64, 128, kernel_size=(3, 3, 3), padding=1),
            nn.MaxPool3d(kernel_size=(2, 2, 2)),
            nn.Linear(128 * 5 * 5 * 6, 128),
            nn.Linear(128, 64),
            nn.Linear(64, 1),
            nn.ReLU(),
            nn.Sigmoid(),
        )

    def forward(self, x):
        return self.convnet(x)
    
model = ConvNet()

# Define the input shape
img_shape = (2, 160, 160, 192)
latent_dim = 100

# Define the generator model
class Generator(nn.Module):
    def __init__(self, latent_dim):
        super(Generator, self).__init__()
        self.fc_layer = nn.Sequential(
            nn.Linear(latent_dim, 163840),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(163840)
        )
        self.conv_layers = nn.Sequential(
            nn.ConvTranspose3d(128, 64, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1), output_padding=(1, 1, 1)),
            nn.BatchNorm3d(64),
            nn.LeakyReLU(0.2),
            nn.ConvTranspose3d(64, 32, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1), output_padding=(1, 1, 1)),
            nn.BatchNorm3d(32),
            nn.LeakyReLU(0.2),
            nn.ConvTranspose3d(32, 2, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1), output_padding=(1, 1, 1)),
            nn.Tanh()
        )

    def forward(self, x):
        x = self.fc_layer(x)
        x = x.view(x.shape[0], 128, 20, 20, 24) # reshaping tensor
        x = self.conv_layers(x)
        return x
    
# Define the discriminator model
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv3d(1, 32, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1)),
            nn.LeakyReLU(0.2),
            nn.Dropout3d(0.25),
            nn.Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1)),
            nn.LeakyReLU(0.2),
            nn.Dropout3d(0.25)
        )
        self.fc_layer = nn.Sequential(
            nn.Linear(163840, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.shape[0], -1)
        x = self.fc_layer(x)
        return x
    
# Create the discriminator and generator models
discriminator = Discriminator()
generator = Generator(latent_dim)

# Define the loss function and optimizer for the discriminator and generator
adversarial_loss = nn.BCELoss()
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_G = torch.optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))

# Define the input shape for the real and fake images
real_imgs = torch.zeros((32, *input_shape))
fake_imgs = torch.zeros((32, latent_dim))

# Train the models
for epoch in range(200):
    # Train the discriminator
    optimizer_D.zero_grad()